In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import basic
import matplotlib.pyplot as plt
import make_wells
import write_inflows
import write_pond_inflows_rch
import Hydrographs
import contextily as ctx
import cartopy.crs as ccrs
import os
from cycler import cycler
import conda_scripts.arich_functions as af
import matplotlib.dates as mdates
import matplotlib.colors as mcolors

from plotly.subplots import make_subplots
from plotly.offline import plot as save_plot
import plotly.graph_objects as go

In [ ]:
ml = basic.load_model()

In [ ]:
sfr = gpd.read_file("SFR_files/only_sfr_cells.shp")

sfr.explore('iseg')

In [ ]:
sfr = gpd.read_file("GIS/")

In [ ]:
basic.basic_map(ml)

In [ ]:
out_folder = 'versions\website_info\wl_trends'

if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [ ]:
datestart = '1/1/2012'
dateend = '1/6/2022'
numdays = (pd.to_datetime(dateend)- pd.to_datetime(datestart)).days



In [ ]:
df = make_wells.load_caissons()
pumping = make_wells.get_period(df, datestart,numdays, assign_per=False)

pumping = pumping.droplevel(1,0).sum(axis=1).to_frame('Total Pumping')/43560.

In [ ]:

start_year = pd.to_datetime(datestart).year

rr = write_inflows.load_riv(station='11464000', title='Russian River', file='RRinflow.dat', figurename='russian_river.png',
              datestart = datestart, out_folder = out_folder, m = ml, numdays=numdays, save_fig=False, write_output=False)

dry = write_inflows.load_riv(station='11465350', title='Dry Creek', file='Dry_creek.dat', figurename='dry_creek.png',
               datestart=datestart, out_folder=out_folder, m=ml, numdays=numdays, save_fig=False, write_output=False)

mw = write_inflows.load_riv(station='11466800', title='Mark West Creek', file='MarkWest.dat', figurename='mark_west.png',
               datestart=datestart, out_folder=out_folder, m=ml, numdays=numdays, save_fig=False, write_output=False)

total = dry.loc[:, 'Q'] + rr.loc[:, 'Q']
total = total.to_frame('rrtotal')

minvalue = 29.54
max_value = 38

stg = write_inflows.load_dam(total, datestart=datestart, minvalue=minvalue, max_value=max_value, numdays=numdays)

In [ ]:
_, pond = write_pond_inflows_rch.load_pond(datestart)
pond = pond.sum(axis = 1).to_frame('Ponds Sum')
# pond = write_pond_inflows_rch.get_period(pond, datestart, numdays)

In [ ]:
# wells_mod = gpd.read_file('GIS/wells_mod.geojson')

wells_mod = Hydrographs.load_wells_mod(ml)

obsall = pd.DataFrame()

for _, wel in wells_mod.iterrows():
    station_name = wel['station_name']
    print(f"plotting {station_name}")
    idx = (0, wel.loc['i_r'], wel.loc['j_c'])
    # head = Hydrographs.get_ts(idx, hdsobj, datestart)
    obs = Hydrographs.load_obs(wel.loc['Filename'], datestart, numdays=numdays)
    
    if obs.shape[0] == 0:
        skip_gw_data = False
    else:
        obs.loc[:,'Well'] = station_name
        obsall = obsall.append(obs.rename(columns={'Value': 'Observed'}))
    
obsall = obsall.set_index('Well',append = True).unstack('Well')
obsall = obsall.droplevel(0,1)
obsall = obsall.loc[:,obsall.notnull().sum().sort_values(ascending = False).index.get_level_values(0)]

In [ ]:
def subc(dfi,axi, colors, name, legend = False, fill = False, ylabel = None, cumulative = False, anomaly = False):
    dfi = dfi.to_frame('WL')
    
    if anomaly:
        dfi = dfi.groupby(dfi.index.month).transform(lambda x: x-x.mean())
    
    dfi.loc[:,'Water Year'] = af.water_year(dfi.index)
    dfi.loc[:,'Date'] = julian_water_year(dfi.index)
        
    for wy, df in dfi.groupby('Water Year'):
        if fill:
            # # axi.fill(df.loc[:,'Date'], df.loc[:,'WL'], c = colors[wy], label = 'WY '+str(wy), alpha = .1)
            # y = df.loc[:,'WL']
            # x = mdates.date2num(df.loc[:,'Date'])
            # # print(x[0:10])
            # print(df.loc[:,'Date'].head())
            # axi.fill_between( x, y,
            #     color = colors[wy], label = 'WY '+str(wy), alpha = .1)
            print('not used')
            
        else:
            if cumulative:
                cplot = df.loc[:,'WL'].cumsum()
            else:
                cplot = df.loc[:,'WL']
                
            axi.plot(df.loc[:,'Date'],cplot, 
                     c = colors[wy], label = 'WY '+str(wy),
                     lw = 1,  markevery=7)
            # else:
            #     axi.plot(df.loc[:,'Date'], df.loc[:,'WL'],
            #              c = colors[wy], 
            #              label = 'WY '+str(wy),
            #              lw = 3)
        
    axi.text(1,1, name, transform = axi.transAxes, ha = 'right', va = 'bottom')
    axi.grid(True)
    if ylabel is None:
        pass
    else:
        axi.set_ylabel(ylabel)
    
    if legend:
        pass

In [ ]:
def julian_water_year(wy):
    '''
    return days from start of water year, creates pseudo date from start of WY 2020.
    Args:
        date: SERIES of dates

    Returns:

    '''
    if isinstance(wy, pd.DatetimeIndex):
        print('converting datetimeindex to df')
        wy.name = 'Date'
        wy = pd.DataFrame(wy, columns=['Date'])
    elif isinstance(wy, pd.Series):
        wy = wy.to_frame('Date')
    elif isinstance(wy, pd.DataFrame):
        print('using index from dataframe to get julian water year date')
        wy = wy.index
        wy.name = 'Date'
        wy = pd.DataFrame(wy, columns=['Date'])
    else:
        raise AssertionError(f"type is not supported {type(wy)}")

    if hasattr(wy, 'month'):
        wy.loc[:, 'month'] = wy.Date.month
        wy.loc[:, 'year'] = wy.Date.year
        wy.loc[:, 'day'] = wy.Date.day
    else:
        wy.loc[:, 'month'] = wy.Date.dt.month
        wy.loc[:, 'year'] = wy.Date.dt.year
        wy.loc[:, 'day'] = wy.Date.dt.day

    c = wy.loc[:, 'month'] < 10
    wy.loc[:, 'year'] = 2000
    wy.loc[~c, 'year'] = 1999
    wy.loc[:, 'WY_date'] = pd.to_datetime(wy.loc[:, ['year', 'month', 'day']])

    wy = wy.loc[:, 'WY_date'].values

    return wy

In [ ]:
def sub_plotly(dfi,fig_pltly, row, name, colors, legend = False, fill = False, ylabel = None, cumulative = False, anomaly = False):
    dfi = dfi.to_frame('WL')
    
    if anomaly:
        dfi = dfi.groupby(dfi.index.month).transform(lambda x: x-x.mean())
    
    dfi.loc[:,'Water Year'] = af.water_year(dfi.index)
    dfi.loc[:,'Date'] = julian_water_year(dfi.index)
        
    for wy, df in dfi.groupby('Water Year'):
        if fill:
            # # axi.fill(df.loc[:,'Date'], df.loc[:,'WL'], c = colors[wy], label = 'WY '+str(wy), alpha = .1)
            # y = df.loc[:,'WL']
            # x = mdates.date2num(df.loc[:,'Date'])
            # # print(x[0:10])
            # print(df.loc[:,'Date'].head())
            # axi.fill_between( x, y,
            #     color = colors[wy], label = 'WY '+str(wy), alpha = .1)
            print('not used')
            
        else:
            if cumulative:
                cplot = df.loc[:,'WL'].cumsum()
            else:
                cplot = df.loc[:,'WL']
                
            fig_pltly.add_trace(go.Scatter(x=df.loc[:,'Date'].values, y=cplot.values,
                                    name = 'WY '+str(wy),
                                    legendgroup = 'wy'+str(wy),
                                    showlegend = legend,
                                          line_color = colors[wy],
                                          ),
              row=row+1, col=1, )
            
            # axi.plot(df.loc[:,'Date'],cplot, 
            #          c = colors[wy], label = 'WY '+str(wy),
            #          lw = 1,  markevery=7)
            # # else:
            #     axi.plot(df.loc[:,'Date'], df.loc[:,'WL'],
            #              c = colors[wy], 
            #              label = 'WY '+str(wy),
            #              lw = 3)
        
    return fig_pltly

#     axi.text(1,1, name, transform = axi.transAxes, ha = 'right', va = 'bottom')
#     axi.grid(True)
#     if ylabel is None:
#         pass
#     else:
#         axi.set_ylabel(ylabel)
    
#     if legend:
#         pass

In [ ]:


# fig = make_subplots(rows=3, cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing=0.02)

# fig.add_trace(go.Scatter(x=[0, 1, 2], y=[10, 11, 12]),
#               row=3, col=1)

# fig.add_trace(go.Scatter(x=[2, 3, 4], y=[100, 110, 120]),
#               row=2, col=1)

# fig.add_trace(go.Scatter(x=[3, 4, 5], y=[1000, 1100, 1200]),
#               row=1, col=1)

# fig.update_layout(height=600, width=600,
#                   title_text="Stacked Subplots with Shared X-Axes")
# fig.show()

In [ ]:
def plot_wl_ptly(wl_obs, river_flow, river_stage, pondinflows, qpump, station, cum = False, anomaly = False):
    
    # fig, ax = plt.subplots(5, sharex = True, gridspec_kw={'height_ratios': [2,1,1,1,1.5]} , figsize = (10, 10))
    fig = make_subplots(rows=5, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02,
                       row_heights = [500]*5,
                 
                       subplot_titles=('Water Levels',  'RR Discharge', 'Dam Elevation (estimated)', 'Pond Inflows',  'Total Pumping'))
    
    
    title = f"{wells_mod.set_index('station_name').loc[:,'station_no'].to_dict()[station]}, {station}"
    # fig.suptitle(title)
    fig.update_layout(title_text=title)
    
    jet = plt.cm.jet
    # n =  wl_obs.loc[:,'Water Year'].nunique()
    wy = pd.Series(af.water_year(wl_obs.index))
    n = wy.nunique()
    colors = [mcolors.to_hex(xx) for xx in  jet(np.linspace(0, 1, n ))]
    # colors = dict(zip( wl_obs.loc[:,'Water Year'].unique(), colors))
    colors = dict(zip( wy.unique(), colors))

        
    fig = sub_plotly(wl_obs.loc[:,station], fig,  0,  'Water Levels', colors, 
                     cumulative = False, anomaly = anomaly, legend = True,)   
    # ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))
    
    fig = sub_plotly(river_flow.iloc[:,0], fig,  1,  'RR Discharge',colors, 
                     cumulative = cum) 
    # subc(river_flow.iloc[:,0], ax[1], colors, 'RR Discharge', cumulative = cum)  
    # ax[1].set_yscale('log')
      
    fig = sub_plotly(river_stage.iloc[:,0].sort_index(), fig,  2,  'Dam Elevation (estimated)',colors,
                     cumulative = False, anomaly = anomaly)
    # subc(river_stage.iloc[:,0].sort_index(), ax[2], colors, 'Dam Elevation (estimated)', cumulative = False)  
    # ax[2].set_ylim([25, 40])
    
    sub_plotly(pondinflows.iloc[:,0], fig,  3,  'Pond Inflows',colors,
               cumulative = cum, anomaly = False)
    # subc(pondinflows.iloc[:,0], ax[3], colors, 'Pond Inflows', ylabel = 'acre-feet', cumulative = cum) 
    
    sub_plotly(qpump.iloc[:,0], fig,  4,  'Total Pumping', colors,
               cumulative = cum, anomaly = False)
    # subc(qpump.iloc[:,0], ax[4], colors, 'Total Pumping', ylabel = 'acre-feet', cumulative = cum) 
    
    
    if cum:
        ex = '_cum.html'
    else:
        ex ='.html'
    if anomaly:
        ex = '_anom'+ex
        
    file = os.path.join(out_folder, station+ex)
    save_plot(fig, filename=file)

    
for col in obsall.filter(regex = 'LRR').columns:
    
    # plot_wl(obsall, total, 
    # stg.loc[:,['INTERP']], 
    # pond.resample('1W').sum(), 
    # pumping.resample('1W').sum(), 
    # col, cum = False ,
    # anomaly = True)

    
    plot_wl_ptly(obsall.resample('3D').mean(), 
                 total.resample('3D').mean(), 
                    stg.loc[:,['INTERP']].resample('3D').mean(), 
                    pond.resample('1W').sum(), 
                    pumping.resample('1W').sum(), 
                    col, cum = False )  
    
    # plot_wl(obsall, total, 
    #     stg.loc[:,['INTERP']], 
    #     pond.resample('1W').sum(), 
    #     pumping.resample('1W').sum(), 
    #     col, cum = False ) 



    
# plot_wl(obsall, total, 
    #         stg.loc[:,['INTERP']], 
    #         pond.resample('1W').sum(), 
    #         pumping.resample('1W').sum(), 
    #         obsall.columns[0] )

In [ ]:
wellnames = wells_mod.set_index('station_name').loc[:,'station_no'].to_dict()

print('''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Water Level trends in observed records</title>
</head>
<body>

</body>
</html>

<h1>Water level trends with Observed Flows/Inflows</h1>''')
for col in obsall.filter(regex = 'LRR').columns:
    print(f"""<a href="wl_trends\{col}.html">Water levels for {col}, ({wellnames[col]})<br></a>""")
    


In [ ]:
def plot_wl(wl_obs, river_flow, river_stage, pondinflows, qpump, station, cum = False, anomaly = False):
    
    fig, ax = plt.subplots(5, sharex = True, gridspec_kw={'height_ratios': [2,1,1,1,1.5]} , figsize = (10, 10))
    title = f"{wells_mod.set_index('station_name').loc[:,'station_no'].to_dict()[station]}, {station}"
    fig.suptitle(title)
    
    #setup colors
    jet = plt.cm.jet
    # n =  wl_obs.loc[:,'Water Year'].nunique()
    wy = pd.Series(af.water_year(wl_obs.index))
    n = wy.nunique()
    colors = jet(np.linspace(0, 1, n ))
    # colors = dict(zip( wl_obs.loc[:,'Water Year'].unique(), colors))
    colors = dict(zip( wy.unique(), colors))
    
    default_cycler = (  cycler(markerfacecolor=(['k','w', 'y']*10)[:n])+
                        cycler(marker=(['s',   'o','*']*10)[:n])+ 
                        cycler(markersize = ([4,4,6]*50)[:n]))
    [ax1.set_prop_cycle(default_cycler) for ax1 in ax]
        
    subc(wl_obs.loc[:,station], ax[0], colors, 'Water Levels', cumulative = False, anomaly = anomaly)   
    ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))
    
    subc(river_flow.iloc[:,0], ax[1], colors, 'RR Discharge', cumulative = cum)  
    ax[1].set_yscale('log')
        
    subc(river_stage.iloc[:,0].sort_index(), ax[2], colors, 'Dam Elevation (estimated)', cumulative = False)  
    ax[2].set_ylim([25, 40])
    
    subc(pondinflows.iloc[:,0], ax[3], colors, 'Pond Inflows', ylabel = 'acre-feet', cumulative = cum) 
    
    subc(qpump.iloc[:,0], ax[4], colors, 'Total Pumping', ylabel = 'acre-feet', cumulative = cum) 
    
    locator = mdates.MonthLocator()
    ax[-1].xaxis.set_major_locator(locator)
    ax[-1].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    
    left = .05
    bottom = .9
    width = .3
    height = .2
    
    axmap = plt.axes([left, bottom, width, height], projection = ccrs.epsg(2226))
    wells_mod.plot(ax = axmap, marker = '.')
    wells_mod.query(f"station_name=='{station}'").plot(ax = axmap, marker = '*',
                                                       markersize= 200.,edgecolor = 'k',
                                                       facecolor = 'w')
    basic.set_bounds(axmap)
    ctx.add_basemap(axmap, crs =2226, attribution = '')
    if cum:
        ex = '_cum.png'
    else:
        ex ='.png'
    if anomaly:
        ex = '_anom'+ex
        
    file = os.path.join(out_folder, station+ex)
    plt.savefig(file, dpi =250, bbox_inches = 'tight')
    print(file)
    
    
for col in obsall.filter(regex = 'LRR').columns:
    
    # plot_wl(obsall, total, 
    # stg.loc[:,['INTERP']], 
    # pond.resample('1W').sum(), 
    # pumping.resample('1W').sum(), 
    # col, cum = False ,
    # anomaly = True)

    
    plot_wl(obsall, total, 
            stg.loc[:,['INTERP']], 
            pond.resample('1W').sum(), 
            pumping.resample('1W').sum(), 
            col, cum = True )  

    plot_wl(obsall, total, 
        stg.loc[:,['INTERP']], 
        pond.resample('1W').sum(), 
        pumping.resample('1W').sum(), 
        col, cum = False ) 



    
# plot_wl(obsall, total, 
    #         stg.loc[:,['INTERP']], 
    #         pond.resample('1W').sum(), 
    #         pumping.resample('1W').sum(), 
    #         obsall.columns[0] )

In [ ]:
wellnames = wells_mod.set_index('station_name').loc[:,'station_no'].to_dict()

print('''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Water Level trends in observed records</title>
</head>
<body>

</body>
</html>

<h1>Water level trends with Observed Flows/Inflows</h1>''')
for col in obsall.filter(regex = 'LRR').columns:
    print(f"""<a href="wl_trends\{col}.png">Water levels for {col}, ({wellnames[col]})<br></a>""")
    
print('''
<h1>Water level trends with Cumulative Observed Flows/Inflows</h1>''')
for col in obsall.filter(regex = 'LRR').columns:
    print(f"""<a href="wl_trends\{col}_cum.png">Water levels for {col}, ({wellnames[col]})<br></a>""")
    
print('''
<h1>Water level trends Monthly Anomaly with Observed Flows/Inflows</h1>''')
for col in obsall.filter(regex = 'LRR').columns:
    print(f"""<a href="wl_trends\{col}_anom.png">Water levels for {col}, ({wellnames[col]})<br></a>""")